# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load libraries and set environment

In [1]:
%cd {'drive/MyDrive/DJIA'}

/content/drive/MyDrive/DJIA


In [2]:
%matplotlib inline
from pathlib import Path

# basic data manipuation
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
from sklearn.preprocessing import MinMaxScaler

# tensorflow utils for neural nets
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Input, concatenate, Embedding, Reshape, BatchNormalization

from tensorflow import keras

# evaluation metrics
from sklearn.metrics import roc_auc_score
from scipy.stats import spearmanr

In [3]:
# install keras-tuner
!pip install -q -U keras-tuner

In [4]:
# hyperparameter tuning bayesian optimization
import keras_tuner as kt
from keras_tuner.tuners import BayesianOptimization

If possible, use GPU, it will enable faster computation when performing optimization of parameters.

In [ ]:
# Check for gpu devices
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


In [5]:
sns.set_style('whitegrid')
np.random.seed(123)

# Load Data

In [6]:
!pip install --upgrade -q tables
data = pd.read_hdf('data/ts_cl.h5', 'all_features')

In [7]:
data.head()

,fwd_returns,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,topic_131,topic_132,topic_133,topic_134,topic_135,topic_136,topic_137,topic_138,topic_139,topic_140,topic_141,topic_142,topic_143,topic_144,topic_145,topic_146,topic_147,topic_148,topic_149,sent_0,sent_1,sent_2,sent_3,sent_4,sent_5,ind_0,ind_1,ind_2,ind_3,ind_4,ind_5,ind_6,ind_7,ind_8,ind_9,ind_10,ind_11,ind_12,ind_13,ind_14
2008-11-04,0.000294,0.004093,-0.011872,-0.009406,0.007194,0.003785,-0.015481,-0.011398,0.006069,0.001119,0.017309,-0.020795,0.002338,0.007854,0.018489,-0.014615,-0.002342,0.001386,-0.029884,0.002925,0.025825,-0.024326,0.003400,0.014623,-0.001025,-0.044167,0.012962,-0.040633,0.038647,0.033463,-0.032731,-0.014663,-0.002672,0.018188,0.010984,-0.069790,0.046810,-0.001805,-0.032150,-0.015022,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.267443,0.561223,0.371533,0.223177,0.258901,0.748283,0.576446,0.243638,0.700463,0.326634,0.562736,0.341358,0.58194,0.886411,0.839742,0.397921,0.358228,0.763304,0.492957
2008-11-05,0.004093,-0.011872,-0.009406,0.007194,0.003785,-0.015481,-0.011398,0.006069,0.001119,0.017309,-0.020795,0.002338,0.007854,0.018489,-0.014615,-0.002342,0.001386,-0.029884,0.002925,0.025825,-0.024326,0.003400,0.014623,-0.001025,-0.044167,0.012962,-0.040633,0.038647,0.033463,-0.032731,-0.014663,-0.002672,0.018188,0.010984,-0.069790,0.046810,-0.001805,-0.032150,-0.015022,-0.035822,...,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,2,0,0.0,1.0,0.396835,0.669851,0.221731,0.141669,0.369867,0.748283,0.715333,0.323243,0.837483,0.423901,0.635387,0.390917,0.58194,0.930801,0.891117,0.455213,0.401954,0.763304,0.601109
2008-11-06,-0.011872,-0.009406,0.007194,0.003785,-0.015481,-0.011398,0.006069,0.001119,0.017309,-0.020795,0.002338,0.007854,0.018489,-0.014615,-0.002342,0.001386,-0.029884,0.002925,0.025825,-0.024326,0.003400,0.014623,-0.001025,-0.044167,0.012962,-0.040633,0.038647,0.033463,-0.032731,-0.014663,-0.002672,0.018188,0.010984,-0.069790,0.046810,-0.001805,-0.032150,-0.015022,-0.035822,-0.051066,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.622076,0.660663,0.126181,0.246742,0.384435,0.748283,0.732322,0.240141,0.844736,0.328708,0.503888,0.326906,0.58194,0.848611,0.901686,0.445698,0.433026,0.763304,0.537385
2008-11-07,-0.009406,0.007194,0.003785,-0.015481,-0.011398,0.006069,0.001119,0.017309,-0.020795,0.002338,0.007854,0.018489,-0.014615,-0.002342,0.001386,-0.029884,0.002925,0.025825,-0.024326,0.003400,0.014623,-0.001025,-0.044167,0.012962,-0.040633,0.038647,0.033463,-0.032731,-0.014663,-0.002672,0.018188,0.010984,-0.069790,0.046810,-0.001805,-0.032150,-0.015022,-0.035822,-0.051066,-0.020007,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.639594,0.707737,0.130854,0.101469,0.504028,0.748283,0.563986,0.132031,0.687746,0.195322,0.407131,0.273090,0.58194,0.668073,0.827244,0.381643,0.427497,0.763304,0.305368
2008-11-10,0.007194,0.003785,-0.015481,-0.011398,0.006069,0.001119,0.017309,-0.020795,0.002338,0.007854,0.018489,-0.014615,-0.002342,0.001386,-0.029884,0.002925,0.025825,-0.024326,0.003400,0.014623,-0.001025,-0.044167,0.012962,-0.040633,0.038647,0.033463,-0.032731,-0.014663,-0.002672,0.018188,0.010984,-0.069790,0.046810,-0.001805,-0.032150,-0.015022,-0.035822,-0.051066,-0.020007,-0.073331,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.369364,0.915475,0.083816,0.127253,0.356919,0.748283,0.719813,0.158778,0.830657,0.221765,0.473513,0.313620,0.58194,0.514885,0.685437,0.312634,0.379074,0.763304,0.389408


Forward returns represent the gained percentage of the DJIA. Hence, we drop it, because what we are trying to predict is precisely the sign of the forward returns. Otherwise, we would be introducing a lot of leackage to our data.

In [8]:
data.drop('fwd_returns', axis=1, inplace=True)

Define the results path for our models.

In [9]:
results_path = Path('results', 'bayes_2lstm')
if not results_path.exists():
    results_path.mkdir(parents=True)

# Define the model

Built stacked model with two and three LSTMs.

In [ ]:
n_features = 1

def build_model(hp):
    '''
    Input:
    ------
    hp: BayesianOptimization object from keras_tuner
    
    Returns:
    -------
    Stacked LSTM model
    '''
    returns = Input(shape=(window, n_features),
                    name='Returns')
    
    topics = Input(shape=(150,),
                   name='Topics')
    
    indicators = Input(shape=(15, ), 
                       name='Indicators')
    
    # neural net units:
    hp_units1 = hp.Int('units1', min_value=10, max_value=100, step=5)
    # hp_units2 = hp.Int('units2', min_value=10, max_value=100, step=5)
    hp_units3 = hp.Int('units3', min_value=10, max_value=100, step=5)

    # dropouts
    hp_dropout1 = hp.Float('dropout1', min_value=0.1, max_value=0.4, sampling='linear')
    # hp_dropout2 = hp.Float('dropout2', min_value=0.1, max_value=0.4, sampling='linear')
    hp_dropout3 = hp.Float('dropout3', min_value=0.1, max_value=0.4, sampling='linear')

    lstm1 = LSTM(units=hp_units1,
                 input_shape=(window,
                          n_features),
             name='LSTM1',
             dropout=hp_dropout1, # 0.2,
             return_sequences=True)(returns)

    # lstm2 = LSTM(units=hp_units2,
    #              input_shape=(window,
    #                       n_features),
    #          name='LSTM2',
    #          dropout=hp_dropout2, # 0.2,
    #          return_sequences=True)(returns)
 
    
    lstm_stack = LSTM(units=hp_units3,
                    dropout=hp_dropout3, # 0.2, 
                    name='LSTM_STACK')(returns)
    
    merged = concatenate([
                          lstm_stack,
                          topics, 
                          indicators
                          # sentiments...
                          ], name='Merged')
    
    bn = BatchNormalization()(merged)
    hp_units4 = hp.Int('units4', min_value=5, max_value=100, step=5)
    hidden_layer = Dense(units=hp_units4, name='Dense')(bn)

    output = Dense(1, name='Output', activation='sigmoid')(hidden_layer)
    rnn_model = Model(inputs=[
                              returns, 
                              topics,
                              indicators
                              ], outputs=output
                      )
    
    # optimizer hyperparams:
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_epsilon = hp.Choice('epsilon', values=[1e-03, 1e-07, 1e-08])

    optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate,
                                         epsilon=hp_epsilon)
    rnn_model.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy',
                               tf.keras.metrics.AUC(name='AUC')
                               ]
                      )
    return rnn_model

# Build Blocking Time Series Split

To avoid data leackage during parameter optimization, we will use blocking time series split.

In [ ]:
# https://hub.packtpub.com/cross-validation-strategies-for-time-series-forecasting-tutorial/

class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits

    def get_n_splits(self, X, y, gropus):
        return self.n_splits
    
    def split(self, X, y=None, groups=None, margin=0):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

# Extend Tuner Class to enable Blocking Time Series Split

In [ ]:
class CVTuner(kt.engine.tuner.Tuner):
    """
    parameters:
    -----------
    window: int
        number of lags for the returns
    extra_inputs: list
        complementary features to the returns we may wish to add. 
        ['topic', 'sent', 'ind']
    epochs: int
        number of epochs
        default: None
        if epochs: None, bayesian search of the best number of epochs
    """
    def __init__(self, 
                 window=60, 
                 extra_inputs=[], 
                 epochs=None,
                 *args, 
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.window = window
        self.extra_inputs = extra_inputs
        self.epochs = epochs


    def build_dataset(self, X, train_indices, test_indices):
        # get rolling sequences
        sequence = list(range(1, self.window+1))
        # get training and test data
        train_data, test_data = X.iloc[train_indices], X.iloc[test_indices]
            
        # Reshape training for neural net
        X_train = [
                # get first window returns
                train_data.loc[:, sequence].values.reshape(-1, window, 1)  
        ]
        y_train = train_data.Label
        
        # Reshape testing for neural net
        X_test = [
                test_data.loc[:, sequence].values.reshape(-1, window, 1)
        ]
        y_test = test_data.Label

        for inp in self.extra_inputs:  # topic, sent, ind
            X_train.append(train_data.filter(like=inp))
            X_test.append(test_data.filter(like=inp))
        
        return (X_train, X_test, y_train, y_test)

    def run_trial(self, trial, X, y, *args, **kwargs):

        tscv = BlockingTimeSeriesSplit(n_splits=5)  # avoid data leackage
        val_accuracy_list, val_auc_list = [], []

        batch_size = trial.hyperparameters.Int('batch_size', 0, 64, step=8)
        if self.epochs is None:
            self.epochs = trial.hyperparameters.Int('epochs', 10, 100, step=10)
        
        for train_indices, test_indices in tscv.split(X):
            # split between train and test
            X_train, X_test, y_train, y_test = self.build_dataset(X, train_indices, test_indices)

            model = self.hypermodel.build(trial.hyperparameters)
            model.fit(X_train, 
                      y_train, 
                      batch_size=batch_size, 
                      epochs=self.epochs)
            
            val_loss, val_accuracy, val_auc = model.evaluate(X_test, y_test)
            val_accuracy_list.append(val_accuracy)
            val_auc_list.append(val_auc)

            self.oracle.update_trial(trial.trial_id, {#'val_accuracy': np.mean(val_accuracy_list)})
                                                       'val_auc':np.mean(val_auc_list)})
            # self.save_model(trial.trial_id, model, step=epochs)

In [ ]:
K.clear_session()

Build tuner

In [ ]:
window = 60
tuner = CVTuner(window=window, 
                extra_inputs=['topic', 'ind'],
                epochs=10,
                oracle=kt.oracles.BayesianOptimization( objective=kt.Objective('val_auc', 'max'),
                                                        max_trials=100,
                                                        num_initial_points=10,
                                                        
                                            ),
                directory='results',
                project_name='bayesian_stacked_lstm',
                hypermodel=build_model,
                overwrite=True
                )

Create Model checkpoint to save the best models.

In [ ]:
lstm_path = (results_path / 'model_bayes_2l_cv.h5').as_posix()

checkpointer = ModelCheckpoint(filepath=lstm_path,
                               verbose=1,
                               monitor='val_AUC',
                               mode='max',
                               save_best_only=True)

Define early stopping. If the model doesn't improve in 5 epochs, stop.

In [ ]:
early_stopping = EarlyStopping(monitor='val_AUC',
                               patience=5,
                               restore_best_weights=True,
                               mode='max')

Split the data between validation and train-test. Validation will be used as out of bag samples and train-test will be created from the rest of the dataset.

In [ ]:
X_data = data.loc[:'2016-03']  # contains all columns, when creating df, label goes to y
y_data = data.Label[:'2016-03']

Bayesian Optimization of hyperparameters

In [ ]:
tuner.search(X_data,
             y_data,
            #  epochs=100,
            #  batch_size=16,
             callbacks=[early_stopping, checkpointer],
             verbose=1)

Results of mean test-set AUC in cross-validation with two and three layers models.

```
Two layers:
===============================================
Trial 56 Complete [00h 00m 33s]
val_auc: 0.5868283790349961

Best val_auc So Far: 0.5868283790349961
Total elapsed time: 00h 40m 26s

Search: Running Trial #57

Hyperparameter    |Value             |Best Value So Far 
units1            |55                |60                
units3            |95                |85                
dropout1          |0.22748           |0.2471            
dropout3          |0.16615           |0.16658           
units4            |5                 |5                 
learning_rate     |0.001             |0.001             
epsilon           |1e-08             |1e-08             
batch_size        |32                |32                

100 Bayesian Optimization iterations with 10 starting points:
Best val_auc So Far: 0.5868283790349961
Total elapsed time: 01h 19m 46s

================================================================================

Three layers:
-------------
==========================================
Best val_auc So Far: 0.5728205786148708
Total elapsed time: 00h 23m 56s

Search: Running Trial #22

Hyperparameter    |Value             |Best Value So Far 
units1            |15                |15                
units2            |85                |20                
units3            |20                |45                
dropout1          |0.20559           |0.35201           
dropout2          |0.1               |0.10089           
dropout3          |0.1               |0.34675           
units4            |100               |60                
learning_rate     |0.0001            |0.0001            
epsilon           |1e-07             |1e-07             
batch_size        |0                 |56                

==========================================
Best val_auc So Far: 0.5576177809635798
Total elapsed time: 00h 56m 25s

Search: Running Trial #52

Hyperparameter    |Value             |Best Value So Far 
units1            |100               |60                
units2            |10                |50                
units3            |80                |85                
dropout1          |0.1               |0.32047           
dropout2          |0.4               |0.39131           
dropout3          |0.4               |0.30969           
units4            |85                |45                
learning_rate     |0.01              |0.001             
epsilon           |1e-08             |0.001             
batch_size        |8                 |48    

100 Bayesian Optimization Iteartions with 10 starting points: 2h
```

The two layer model actually performs better on the cross validation. Getting about 1% better in the test-AUC and taking almost half the time.

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)

Save the best model found by Bayesian Optimization.

In [ ]:
model.save('results/bayes_lstm/bayes_5868.h5')

# Train the model

In [65]:
model = keras.models.load_model('results/bayes_lstm/bayes_5868.h5')

In [66]:
train_test_data = data.loc[:'2016-03']  # contains all columns, when creating df, label goes to y

In [67]:
from src.ts_utils.rnn import stacked_LSTM

In [68]:
lstm_split = stacked_LSTM(df=train_test_data, inputs=['topic', 'ind'])

In [69]:
X_train, X_test, y_train, y_test = lstm_split.build_lstm_dataset()

In [70]:
lstm_path = (results_path / 'bayes_5868_train.h5').as_posix()

checkpointer = ModelCheckpoint(filepath=lstm_path,
                               verbose=1,
                               monitor='val_AUC',
                               mode='max',
                               save_best_only=True)

early_stopping = EarlyStopping(monitor='val_AUC',
                               patience=5,
                               restore_best_weights=True,
                               mode='max')

history = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, checkpointer],
                    verbose=1)

Epoch 1/100
47/47 [==============================] - ETA: 0s - loss: 0.8108 - accuracy: 0.5228 - AUC: 0.5276
Epoch 00001: val_AUC improved from -inf to 0.51281, saving model to results/bayes_2lstm/bayes_5868_train.h5
47/47 [==============================] - 33s 63ms/step - loss: 0.8108 - accuracy: 0.5228 - AUC: 0.5276 - val_loss: 0.7178 - val_accuracy: 0.5121 - val_AUC: 0.5128
Epoch 2/100
47/47 [==============================] - ETA: 0s - loss: 0.7541 - accuracy: 0.5362 - AUC: 0.5404
Epoch 00002: val_AUC did not improve from 0.51281
47/47 [==============================] - 2s 39ms/step - loss: 0.7541 - accuracy: 0.5362 - AUC: 0.5404 - val_loss: 0.7227 - val_accuracy: 0.5255 - val_AUC: 0.5045
Epoch 3/100
46/47 [============================>.] - ETA: 0s - loss: 0.7183 - accuracy: 0.5577 - AUC: 0.5646
Epoch 00003: val_AUC did not improve from 0.51281
47/47 [==============================] - 2s 40ms/step - loss: 0.7187 - accuracy: 0.5584 - AUC: 0.5640 - val_loss: 0.7256 - val_accuracy: 0.5

### Get the validation set data

In [19]:
val_data = data.loc['2016-04':]  # leave june as out of sample testing

In [21]:
window=60
sequence = list(range(1, window+1))
X_val = [
           val_data.loc[:, sequence].values.reshape(-1, window, 1),  # get first window returns
           val_data.filter(like='topic'),  # get main news topics
        #    val_data.filter(like='sent'),   # sentiment from news
           val_data.filter(like='ind')     # indicators from ts
]
y_val = val_data.Label

#### Validation *58.68* on CV

In [22]:
model.evaluate(X_val, y_val)

3/3 [==============================] - 0s 9ms/step - loss: 0.6981 - accuracy: 0.4769 - AUC: 0.5236


[0.6980962157249451, 0.4769230782985687, 0.5236486196517944]

AUC of 52.36, better than random for bet-sizing.

#### Validation *55.76* on CV

In [64]:
model.evaluate(X_val, y_val)

3/3 [==============================] - 0s 16ms/step - loss: 0.7750 - accuracy: 0.6000 - AUC: 0.5174


[0.7749996185302734, 0.6000000238418579, 0.5173745155334473]

The accuracy achieved gets to 60% and the AUC is of 51.73%. Hence, being better results than random.

#### Always buy method:

In [80]:
(np.ones(len(y_val)) == y_val).sum()/len(y_val)

0.5692307692307692

We would have achieved almost a 57% accuracy, 3% worse than the one we got from our model. 

#### Complete random method

In [99]:
import random
(random.choices([0, 1], k=len(y_val)) == y_val).sum()/len(y_val)

0.46153846153846156

Method gets below 50%, however, on average, we would expect it to be around 50%. (In this case, it would be lower, due to the fact that almost 57% of the values are ones.)